# The Adding Problem

### Mehrdad Yazdani
### September 22, 2018

Colab notebook online to play!!


https://colab.research.google.com/drive/1TEDCmXo2ZqzExZZmpYKUdkkv0xwff0N5



The "adding problem" was original proposed by Schmidhuber and colleagues as an example of a sequential task that LSTM's are particularly well suited for: http://people.idsia.ch/~juergen/nipslstm/node4.html


>![The Adding Problem](https://minpy.readthedocs.io/en/latest/_images/adding_problem.png)

As another example, the following sequence of length 5
```
{(0.443, 0), 
 (0.112, 1), 
 (0.950, 0), 
 (0.839, 1), 
 (0.142, 0)} 
 ```

yields 0.112 + 0.839 = 0.951 as the answer since the 2nd and 4th elements are added. 



Here we will compare several different cells in PyTorch to see how well they solve the adding problem. The cells we consider are:

- RNN
- LSTM
- RNN with identity initialization

We will also consider a convolutional layer. Conv1D is not a recurrent layer, but has been shown to me useful for some sequential tasks. 

All methods will be compared using MSE on a held out test set. 

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pylab as plt
import seaborn as sns;
%matplotlib inline

In [2]:
import sys
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [3]:
# use CUDA or not
use_cuda = False
if use_cuda and torch.cuda.is_available():
  print("using cuda!")
  device = torch.device("cuda")
else:
  print("using CPU!")

using CPU!


## Data loading functions

We will define some helper functions to generate our datasets. `generate_sequence` will genrate a single sequence whereas `get_set` returns multiple sequences (so a *dataset* of sequences).



In [4]:
def generate_sequence(seq_len = 10):
  ''' generate sequences
  
  Args:
  -----
  seq_len : int (default 10)
    The length of the sequence
  
  Returns:
  --------
  tuple of 3 numpy arrays x, z, y. x and z are 1D arrays and have same length
  y is a float that is the target we want to predict (addition of x masked by z)
  
  Example:
  --------
  
  >>> x_seq, z_seq, y_target = generate_sequence(seq_len = 100)
  
  '''
  x = np.random.rand(seq_len)
  z_p = np.arange(seq_len)
  np.random.shuffle(z_p)
  z = np.zeros(seq_len)
  z[z_p[0]] = 1
  z[z_p[1]] = 1
  y = x[z_p[0]] + x[z_p[1]]
  return x, z, y

def get_set(num_examples = 100, seq_len = 10):
  '''
  Get the data set used for training/testing networks.
  
  Args:
  -----
  num_examples : int (default 100)
    Number of sequences to generate
  
  seq_len : int (default 10)
    The length of the sequence
    
  Returns:
  --------
  typle of length 2 where the first tuple is a numpy array of shape 
  num_examples x seq_len x 2 and the second tuple is length num_examples
  
  Example:
  --------
  
  >>> X, y = get_set(num_examples=1000, seq_len = 50)
  
  '''
  X_set, Z_set, y_set = [], [], []

  for _ in range(num_examples):
    x_example, z_example, y_example = generate_sequence(seq_len)
    X_set.append(x_example)
    Z_set.append(z_example)
    y_set.append(y_example)
    
  X = np.zeros((num_examples,seq_len,2))
  X[:,:,0] = np.array(X_set)
  X[:,:,1] = np.array(Z_set)
  return X, np.array(y_set)  

Lets see `get_set` in action:

In [5]:
X_train, y_train = get_set(num_examples=100, seq_len = 10)
X_test, y_test = get_set(num_examples=100, seq_len = 10)


So for the input we have a 3D array that has shape "num examples" x "sequence length" x "num features."


Note that the datasets that `get_set` returns are Numpy arrays, but PyTorch recquires PyTorch tensors. We could of course convert these Numpy arrays to PyTorch arrays, and then do some booking with indices to keep track of going through different batches when doing batch updates on the network.

But that is tedious and PyTorch offers the Dataset class that we can inherit from to keep all this bookkeeping for us. Below we define the `SequenceDataset` generator class that will be used for all our data handilng for PyTorch. 

In [6]:
class SequenceDataset(Dataset):
  
  def __init__(self, num_examples, seq_len):
    self.num_examples = num_examples
    self.seq_len = seq_len
    
    X, y = get_set(num_examples=self.num_examples, seq_len = self.seq_len)
    self.X = torch.from_numpy(X).float()
    self.y = torch.from_numpy(y).float()
    if use_cuda and torch.cuda.is_available():
      self.X = self.X.to(device)
      self.y = self.y.to(device)
    
    
    
  def __getitem__(self, index):
    return self.X[index], self.y[index]
  
  def __len__(self):
    return self.num_examples

  

Lets create a training and test set with 100 examples for each and sequence lengths of 10. 

In [7]:
train_set = SequenceDataset(num_examples=100, seq_len = 10)
test_set = SequenceDataset(num_examples=100, seq_len = 10)



We can use PyTorch's `DataLoader` to specify the the batches of data to load for training. Note that each of the 100 example sequences are independent, so we also shuffle the order of the different sequences. 


In [8]:
batch_size = 32

train_loader = DataLoader(dataset = train_set,
                          batch_size=batch_size,
                          shuffle = True)

test_loader = DataLoader(dataset = test_set,
                         batch_size=batch_size,
                         shuffle = True)

## RNN

We will start solving the Adding Problem with a simple RNN (the *Elman Network*). The network will update its internal hidden state for every element in the sequence until we reach the end. When we reach the end, we pass the final hidden state through a fully connected linear layer to predict the target. This type of architecture is sometimes called *many-to-one* since we are taking "many" elements (a sequence) to a single element (the target).

<center>
![Many to one](https://i.stack.imgur.com/QCnpU.jpg)
</center>

In [9]:
class RNNAdder(nn.Module):

    def __init__(self, hidden_size, input_size):    
        super(RNNAdder, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size 
        
        self.rnn = nn.RNN(input_size=self.input_size,
                          hidden_size=self.hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Initialize hidden state. The shape of the tensor is
        # (num_layers * num_directions, batch, hidden_size)
        h_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        # since our input has the batch dimension in the first dim, 
        # we just use x.size(0)        
        if use_cuda and torch.cuda.is_available():
          h_0 = h_0.to(device)

        # Propagate input through RNN
        # Input: (batch, seq_len, embedding_size)
        _, h_f = self.rnn(x, h_0)
        # we only care about the final hidden state. The intermediate values 
        # of the hidden state are discarded. We pass the final hidden state
        # through the fully connected linear layer
        return self.fc(h_f).squeeze()


In [10]:
rnn_adder = RNNAdder(hidden_size = 12, input_size = 2)

if use_cuda and torch.cuda.is_available():
    rnn_adder = rnn_adder.cuda(device)

In [11]:
# Set loss and optimizer function
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(rnn_adder.parameters(), lr=0.01)

In [12]:
%%time
num_epochs = 1000
for epoch in range(num_epochs):
  for i, (sequences, targets) in enumerate(train_loader):
    
    # forward pass
    outputs = rnn_adder(sequences)
    loss = criterion(outputs, targets)
    
    # update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  if (epoch+1)%100 == 0:
    print("loss is", loss.item())

loss is 0.0587642639875412
loss is 0.004322755616158247
loss is 0.011347895488142967
loss is 0.0040663788095116615
loss is 0.01571684516966343
loss is 0.00023949150636326522
loss is 0.0006890100194141269
loss is 0.002038390375673771
loss is 0.004899691324681044
loss is 0.0003933361731469631
CPU times: user 6.28 s, sys: 66.1 ms, total: 6.34 s
Wall time: 6.34 s


In [13]:
with torch.no_grad():
  outputs = rnn_adder(test_set.X)
  test_mse = torch.mean((outputs - test_set.y)**2)
print(test_mse.item())

0.10881263762712479


## LSTM

RNN's suffer from the vanishing gradient problem since creating the final hidden state is a result of updating the state through multiplications everytime a new element arrives in the sequence. LSTM's bypass this challenge by updating state additively. As a result, updaing gradients is much easier and longer memories can persist. Below is an `LSTMAdder` that is nearly identical to the `RNNAdder.`



In [14]:
class LSTMAdder(nn.Module):

    def __init__(self, hidden_size, input_size):    
        super(LSTMAdder, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size 
        self.lstm = nn.LSTM(input_size=self.input_size,
                          hidden_size=self.hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        h_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        if use_cuda and torch.cuda.is_available():
          h_0 = h_0.to(device)
          c_0 = c_0.to(device)

        # Propagate input through LSTM
        # Input: (batch, seq_len, embedding_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)
        _, (h_f, c_f) = self.lstm(x, (h_0, c_0))
        return self.fc(h_f).squeeze()


In [15]:
lstm_adder = LSTMAdder(hidden_size = 12, input_size = 2)
if use_cuda and torch.cuda.is_available():
    lstm_adder = lstm_adder.cuda(device)

In [16]:
# Set loss and optimizer function
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm_adder.parameters(), lr=0.01)

In [17]:
%%time
num_epochs = 1000
for epoch in range(num_epochs):
  for i, (sequences, targets) in enumerate(train_loader):
    # forward pass
    outputs = lstm_adder(sequences)
    loss = criterion(outputs, targets)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  if (epoch+1)%100 == 0:
    print("loss is", loss.item())

loss is 0.013530051335692406
loss is 0.0017925265710800886
loss is 0.0006940184393897653
loss is 0.0006754589267075062
loss is 0.0002635184209793806
loss is 0.0001562008837936446
loss is 0.00045400409726426005
loss is 0.00019081193022429943
loss is 0.00014291123079601675
loss is 0.00011090301995864138
CPU times: user 12.8 s, sys: 113 ms, total: 12.9 s
Wall time: 13 s


In [18]:
with torch.no_grad():
  outputs = lstm_adder(test_set.X)
  test_mse = torch.mean((outputs - test_set.y)**2)
print(test_mse.item())

0.000431636581197381


## ReLU RNN

The idea of the ReLU RNN is to initialize the hidden state of the RNN with the identity matrix and the bias with 0 and use the ReLU activation function. Below we demonstrate how such an RNN can be implemented. The results are not as good as the LSTM but certainly better than the traditional Elman Network.

In [19]:
class ReLURNNAdder(nn.Module):

    def __init__(self, hidden_size, input_size):    
        super(ReLURNNAdder, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size 
        
        self.rnn = nn.RNN(input_size=self.input_size,
                          hidden_size=self.hidden_size, nonlinearity = "relu",
                          batch_first=True)
        
        torch.nn.init.zeros_(self.rnn.weight_ih_l0)
        torch.nn.init.eye_(self.rnn.weight_hh_l0)
        
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Initialize hidden state. The shape of the tensor is
        # (num_layers * num_directions, batch, hidden_size)
        h_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        # since our input has the batch dimension in the first dim, 
        # we just use x.size(0)        
        if use_cuda and torch.cuda.is_available():
          h_0 = h_0.to(device)

        # Propagate input through RNN
        # Input: (batch, seq_len, embedding_size)
        _, h_f = self.rnn(x, h_0)
        # we only care about the final hidden state. The intermediate values 
        # of the hidden state are discarded. We pass the final hidden state
        # through the fully connected linear layer
        return self.fc(h_f).squeeze()


We could train this model as before but if we want to be fair in our comparisons,  we should train each adder for each epoch for each batch. This can help us control the differences in training procedures. 

In [20]:
relu_rnn_adder = ReLURNNAdder(hidden_size = 12, input_size = 2)
rnn_adder = RNNAdder(hidden_size = 12, input_size = 2)
lstm_adder = LSTMAdder(hidden_size = 12, input_size = 2)


if use_cuda and torch.cuda.is_available():
    relu_rnn_adder = relu_rnn_adder.cuda(device)
    rnn_adder = rnn_adder.cuda(device)
    lstm_adder = lstm_adder.cuda(device)

In [22]:
# Set loss and optimizer function
criterion = torch.nn.MSELoss()
relu_rnn_opt = torch.optim.Adam(relu_rnn_adder.parameters(), lr=0.01)
rnn_opt = torch.optim.Adam(rnn_adder.parameters(), lr=0.01)
lstm_opt = torch.optim.Adam(lstm_adder.parameters(), lr=0.01)


In [23]:
def update_model(model, optimizer, input_sequences, output_targets):
  preds = model(input_sequences)
  loss = criterion(preds, output_targets)
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  return loss.item()

In [24]:
%%time
lstm_losses = []
rnn_losses = []
relu_rnn_losses = []
for epoch in range(5000):
  for i, (sequences, targets) in enumerate(train_loader):
    
    loss = update_model(relu_rnn_adder, relu_rnn_opt, sequences, targets)
    relu_rnn_losses.append(loss)
    
    loss = update_model(rnn_adder, rnn_opt, sequences, targets)
    rnn_losses.append(loss)

    loss = update_model(lstm_adder, lstm_opt, sequences, targets)
    lstm_losses.append(loss)

    
  if (epoch+1)%100 == 0:
    print("LSTM loss:{:.2e}".format(lstm_losses[-1]) , 
          "RNN loss:{:.2e}".format(rnn_losses[-1]), 
          "ReLURNN loss:{:.2e}".format(relu_rnn_losses[-1]))

LSTM loss:3.61e-02 RNN loss:1.24e-01 ReLURNN loss:2.67e-01
LSTM loss:5.37e-03 RNN loss:2.56e-02 ReLURNN loss:1.56e-01
LSTM loss:1.43e-03 RNN loss:1.07e-02 ReLURNN loss:4.33e-02
LSTM loss:3.94e-04 RNN loss:4.47e-03 ReLURNN loss:1.39e-01
LSTM loss:1.12e-03 RNN loss:3.70e-03 ReLURNN loss:1.74e-01
LSTM loss:9.05e-04 RNN loss:9.02e-03 ReLURNN loss:1.18e-01
LSTM loss:1.26e-04 RNN loss:1.28e-03 ReLURNN loss:3.04e-02
LSTM loss:8.55e-05 RNN loss:8.85e-04 ReLURNN loss:2.19e-01
LSTM loss:4.08e-04 RNN loss:2.74e-02 ReLURNN loss:1.43e-01
LSTM loss:2.89e-04 RNN loss:7.51e-04 ReLURNN loss:4.85e-02
LSTM loss:1.16e-04 RNN loss:1.14e-03 ReLURNN loss:2.76e-01
LSTM loss:2.82e-04 RNN loss:3.63e-04 ReLURNN loss:3.75e-01
LSTM loss:9.90e-05 RNN loss:1.18e-03 ReLURNN loss:1.82e-01
LSTM loss:3.01e-04 RNN loss:5.72e-03 ReLURNN loss:2.79e-01
LSTM loss:8.01e-05 RNN loss:2.23e-04 ReLURNN loss:6.04e-02
LSTM loss:1.00e-04 RNN loss:1.12e-03 ReLURNN loss:7.01e-02
LSTM loss:4.11e-05 RNN loss:8.61e-04 ReLURNN loss:1.51e-

In [25]:
with torch.no_grad():
  outputs = relu_rnn_adder(test_set.X)
  relu_rnn_mse = torch.mean((outputs - test_set.y)**2)

  outputs = rnn_adder(test_set.X)
  rnn_mse = torch.mean((outputs - test_set.y)**2)

  outputs = lstm_adder(test_set.X)
  lstm_mse = torch.mean((outputs - test_set.y)**2)
  
  


In [26]:
lstm_mse.item(), rnn_mse.item(), relu_rnn_mse.item()

(0.0004923116648569703, 0.09761270880699158, 0.12372004240751266)

While the LSTM is still the superior adder, the RNN initialized with the identity matrix and using the ReLU function is definitely better than the traidtional RNN.
